# Определение стоимости автомобилей

## Подготовка данных

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
from sklearn.model_selection import RepeatedKFold
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import OrdinalEncoder

In [3]:
try:
    df = pd.read_csv('/datasets/autos.csv')
except:
    df = pd.read_csv('C:/documents/datasets/autos.csv')

In [4]:
display(df.head(5))
display(df.info())
display(df.describe())

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [6]:
df['DateCrawled'] = pd.to_datetime(df['DateCrawled'])
df['DateCreated'] = pd.to_datetime(df['DateCreated'])
df['LastSeen'] = pd.to_datetime(df['LastSeen'])
display(df['DateCrawled'].describe())
display(df['DateCreated'].describe())
display(df['LastSeen'].describe())

count                  354369
unique                 271174
top       2016-03-24 14:49:47
freq                        7
first     2016-03-05 14:06:22
last      2016-04-07 14:36:58
Name: DateCrawled, dtype: object

count                  354369
unique                    109
top       2016-04-03 00:00:00
freq                    13719
first     2014-03-10 00:00:00
last      2016-04-07 00:00:00
Name: DateCreated, dtype: object

count                  354369
unique                 179150
top       2016-04-06 13:45:54
freq                       17
first     2016-03-05 14:15:08
last      2016-04-07 14:58:51
Name: LastSeen, dtype: object

In [7]:
df['Model'] = df['Model'].fillna('unknown')
df['Gearbox'] = df['Gearbox'].fillna('unknown')
df['VehicleType'] = df['VehicleType'].fillna('unknown')
df['FuelType'] = df['FuelType'].fillna('unknown')
df['Repaired'] = df['Repaired'].fillna('unknown')

In [8]:
df = df.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'RegistrationMonth', 'NumberOfPictures', 'PostalCode'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354369 non-null  int64 
 1   VehicleType       354369 non-null  object
 2   RegistrationYear  354369 non-null  int64 
 3   Gearbox           354369 non-null  object
 4   Power             354369 non-null  int64 
 5   Model             354369 non-null  object
 6   Kilometer         354369 non-null  int64 
 7   FuelType          354369 non-null  object
 8   Brand             354369 non-null  object
 9   Repaired          354369 non-null  object
dtypes: int64(4), object(6)
memory usage: 27.0+ MB


In [10]:
df.describe()

,Price,RegistrationYear,Power,Kilometer
count,354369.000000,354369.000000,354369.000000,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535
std,4514.158514,90.227958,189.850405,37905.341530
min,0.000000,1000.000000,0.000000,5000.000000
25%,1050.000000,1999.000000,69.000000,125000.000000
50%,2700.000000,2003.000000,105.000000,150000.000000
75%,6400.000000,2008.000000,143.000000,150000.000000
max,20000.000000,9999.000000,20000.000000,150000.000000


In [9]:
df = df[df['Price'] > 0]

In [10]:
df = df[(df['RegistrationYear'] > 1900) 
        & (df['RegistrationYear'] <= 2016)]

In [11]:
df = df[(df['Power'] > 0) & (df['Power'] < df['Power'].quantile(.99))]
display(df.describe())
df.info()

,Price,RegistrationYear,Power,Kilometer
count,293415.000000,293415.000000,293415.000000,293415.000000
mean,4779.405838,2002.817075,118.183433,128197.757443
std,4548.594609,6.553088,48.692248,36843.054375
min,1.000000,1910.000000,1.000000,5000.000000
25%,1300.000000,1999.000000,75.000000,125000.000000
50%,3100.000000,2003.000000,110.000000,150000.000000
75%,6900.000000,2007.000000,147.000000,150000.000000
max,20000.000000,2016.000000,299.000000,150000.000000


<class 'pandas.core.frame.DataFrame'>
Int64Index: 293415 entries, 1 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             293415 non-null  int64 
 1   VehicleType       293415 non-null  object
 2   RegistrationYear  293415 non-null  int64 
 3   Gearbox           293415 non-null  object
 4   Power             293415 non-null  int64 
 5   Model             293415 non-null  object
 6   Kilometer         293415 non-null  int64 
 7   FuelType          293415 non-null  object
 8   Brand             293415 non-null  object
 9   Repaired          293415 non-null  object
dtypes: int64(4), object(6)
memory usage: 24.6+ MB


Был загружен датасет с 16 параметрами. Параметры VehicleType, Gearbox, Model, FuelType, Repaired - содержат пропуски. Они образовались из-за технической ошибки при выгрузке или человеческий фактор - были пропущены при заполнении анкеты. Мы приняли решение создать доп. столбец - месяц продажи, тк на цену может оказывать влияние сезонность. Данные выгружены с марта 2014 по апрель 2016 года. Столбцы 'DateCrawled', 'DateCreated', 'LastSeen', 'RegistrationMonth', 'NumberOfPictures' - считаем не информативными и не влияющими на стоимость автомобиля, поэтому они были удалены. Так же в связи с невозможность восстановить - были заменены на unknown строки с пропусками по столбцам VehicleType, Gearbox, Model, FuelType, Repaired - тк они могу оказывать существенное влияние на цену. Так же были удалены 1-3 и 99 квантили в данных, тк наблюдались аномалии в параметрах 'Price', 'RegistrationYear' и 'Power'. После подготовки в датафрейме осталось 293415 строк (83% исходных данных)

## Обучение моделей

In [12]:
def rmse(predictions, targets):
    return np.sqrt(np.mean((predictions - targets) ** 2))

In [13]:
scorer = make_scorer(rmse, greater_is_better=False)

In [14]:
features = df.drop(['Price'], axis=1)
target = df['Price']

features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, test_size=0.25, random_state=12345)
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(220061, 9)
(73354, 9)
(220061,)
(73354,)


In [15]:
col= ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']
encoder = OrdinalEncoder()

In [40]:
encoder.fit(features[col])

features_train_o=features_train.copy()
features_train_o[col]= pd.DataFrame(encoder.transform(features_train[col]), columns = features_train[col].columns)

features_test_o = features_test.copy()
features_test_o[col]= pd.DataFrame(encoder.transform(features_test[col]), columns = features_test[col].columns)

target_train_o = target_train
target_test_o = target_test

print(features_train_o.shape)
print(features_test_o.shape)
print(target_train_o.shape)
print(target_test_o.shape)

features_train_oh = pd.get_dummies(features_train, columns=['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired'], drop_first=True)
features_test_oh = pd.get_dummies(features_test, columns=['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired'], drop_first=True)

(220061, 9)
(73354, 9)
(220061,)
(73354,)


In [41]:
for i in range(len(features_train_oh.columns)):
    a=0
    if features_train_oh.columns[i] != features_test_oh.columns[i]:
        print('Ошибка в стольце:', i)

Ошибка в стольце: 126
Ошибка в стольце: 127
Ошибка в стольце: 128
Ошибка в стольце: 129
Ошибка в стольце: 130
Ошибка в стольце: 131
Ошибка в стольце: 132
Ошибка в стольце: 133
Ошибка в стольце: 134
Ошибка в стольце: 135
Ошибка в стольце: 136
Ошибка в стольце: 137
Ошибка в стольце: 138
Ошибка в стольце: 139
Ошибка в стольце: 140
Ошибка в стольце: 141
Ошибка в стольце: 142
Ошибка в стольце: 143
Ошибка в стольце: 144
Ошибка в стольце: 145
Ошибка в стольце: 146
Ошибка в стольце: 147
Ошибка в стольце: 148
Ошибка в стольце: 149
Ошибка в стольце: 150
Ошибка в стольце: 151
Ошибка в стольце: 152
Ошибка в стольце: 153
Ошибка в стольце: 154
Ошибка в стольце: 155
Ошибка в стольце: 156
Ошибка в стольце: 157
Ошибка в стольце: 158
Ошибка в стольце: 159
Ошибка в стольце: 160
Ошибка в стольце: 161
Ошибка в стольце: 162
Ошибка в стольце: 163
Ошибка в стольце: 164
Ошибка в стольце: 165
Ошибка в стольце: 166
Ошибка в стольце: 167
Ошибка в стольце: 168
Ошибка в стольце: 169
Ошибка в стольце: 170
Ошибка в с

IndexError: index 304 is out of bounds for axis 0 with size 304

In [18]:
%%time
model_lr = LinearRegression()
model_lr.fit(features_train_oh, target_train_o)
rmse_lr = cross_val_score(model_lr, features_train_oh, target_train_o, cv=5, scoring=scorer).mean()
print('Лучшее значение remse при линейной регрессии на трейне:', -rmse_lr)

Лучшее значение remse при линейной регрессии на трейне: 2561.5113542745744
CPU times: user 1min 41s, sys: 1min 24s, total: 3min 5s
Wall time: 3min 5s


In [50]:
%%time
model_lr.fit(features_train_oh, target_train_o)

CPU times: user 15.2 s, sys: 4.22 s, total: 19.5 s
Wall time: 19.5 s


LinearRegression()

In [27]:
%%time
pred = model_lr.predict(features_train_oh)

NameError: name 'model_lr' is not defined

In [20]:
a = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']
model_cb = CatBoostRegressor(random_state = 12345, cat_features = a)
parameters = {'depth' : [6,8,10],
              'learning_rate' : [0.01, 0.05, 0.1],
              'iterations'    : [30, 50, 100]
              }

grid = GridSearchCV(estimator=model_cb, param_grid = parameters, cv = 5, n_jobs=-1, scoring=scorer)
grid.fit(features_train, target_train)

print('Лучшее значение rmse и параметров CatBoostRegressor на тесте:',
      -grid.best_score_, grid.best_params_)

0:	learn: 4520.3404685	total: 161ms	remaining: 4.66s
1:	learn: 4490.8114855	total: 257ms	remaining: 3.59s
2:	learn: 4461.6073875	total: 349ms	remaining: 3.14s
3:	learn: 4432.0702673	total: 431ms	remaining: 2.8s
4:	learn: 4402.9570399	total: 508ms	remaining: 2.54s
5:	learn: 4373.9460590	total: 596ms	remaining: 2.38s
6:	learn: 4345.8627494	total: 672ms	remaining: 2.21s
7:	learn: 4317.7316069	total: 741ms	remaining: 2.04s
8:	learn: 4289.9344648	total: 828ms	remaining: 1.93s
9:	learn: 4262.3470179	total: 911ms	remaining: 1.82s
10:	learn: 4235.6717039	total: 999ms	remaining: 1.72s
11:	learn: 4209.2370681	total: 1.08s	remaining: 1.61s
12:	learn: 4182.4165510	total: 1.15s	remaining: 1.5s
13:	learn: 4156.1080869	total: 1.23s	remaining: 1.4s
14:	learn: 4130.1508778	total: 1.3s	remaining: 1.3s
15:	learn: 4104.7792044	total: 1.41s	remaining: 1.23s
16:	learn: 4080.2798613	total: 1.5s	remaining: 1.15s
17:	learn: 4055.3248373	total: 1.61s	remaining: 1.07s
18:	learn: 4031.0147413	total: 1.69s	remaini

In [21]:
%%time
model_cb = CatBoostRegressor(random_state = 12345, cat_features = a, depth=10, iterations=100, learning_rate=0.1)
model_cb.fit(features_train, target_train)

0:	learn: 4223.1357104	total: 161ms	remaining: 15.9s
1:	learn: 3936.0296386	total: 315ms	remaining: 15.5s
2:	learn: 3683.1749037	total: 491ms	remaining: 15.9s
3:	learn: 3462.0625776	total: 663ms	remaining: 15.9s
4:	learn: 3268.3135082	total: 874ms	remaining: 16.6s
5:	learn: 3096.9473903	total: 1.02s	remaining: 15.9s
6:	learn: 2938.1997376	total: 1.18s	remaining: 15.7s
7:	learn: 2804.7759467	total: 1.34s	remaining: 15.4s
8:	learn: 2687.5846346	total: 1.52s	remaining: 15.3s
9:	learn: 2583.6606643	total: 1.68s	remaining: 15.1s
10:	learn: 2494.0221201	total: 1.83s	remaining: 14.8s
11:	learn: 2404.8732947	total: 1.98s	remaining: 14.5s
12:	learn: 2325.8029209	total: 2.14s	remaining: 14.3s
13:	learn: 2259.7984622	total: 2.29s	remaining: 14.1s
14:	learn: 2200.0823948	total: 2.45s	remaining: 13.9s
15:	learn: 2146.7574503	total: 2.59s	remaining: 13.6s
16:	learn: 2102.6197312	total: 2.74s	remaining: 13.4s
17:	learn: 2063.2003437	total: 2.88s	remaining: 13.1s
18:	learn: 2021.5495262	total: 3.04s	r

In [22]:
%%time
pr = model_cb.predict(features_train)

CPU times: user 444 ms, sys: 55 µs, total: 444 ms
Wall time: 449 ms


In [24]:
model_LGB = LGBMRegressor()

In [28]:
params = [{'n_estimators':[100, 500, 1000], 
            'learning_rate':[0.1, 0.3, 0.5],
            'random_state':[12345]}]

grid1 = GridSearchCV(model_LGB, params, scoring=scorer, cv = 5)
grid1.fit(features_train_o, target_train_o)
print(grid1.best_params_)
print(-grid.best_score_)

{'learning_rate': 0.1, 'n_estimators': 500, 'random_state': 12345}
1630.3094803840784


In [29]:
%%time
model_LGB = LGBMRegressor(n_estimators=500, random_state=12345, learning_rate=0.1, objective='rmse')
model_LGB.fit(features_train_o, target_train_o, eval_set=(features_test_o, target_test_o), verbose=100)

[100]	valid_0's rmse: 2041.51
[200]	valid_0's rmse: 2024.41
[300]	valid_0's rmse: 2016.68
[400]	valid_0's rmse: 2011.36
[500]	valid_0's rmse: 2008.53
CPU times: user 18.8 s, sys: 287 ms, total: 19.1 s
Wall time: 19.3 s


LGBMRegressor(n_estimators=500, objective='rmse', random_state=12345)

In [30]:
%%time
predict_LGB= model_LGB.predict(features_train_o)

CPU times: user 6.46 s, sys: 109 ms, total: 6.57 s
Wall time: 6.62 s


Были обучены и найдены наилучшие гиперпараметры для 3-х моделей: Линейная регрессия, LGBMRegressor и CatBoostRegressor

## Анализ моделей

При проверке моделей при кросс валидации и поиске лучших гиперпарметров на тренировочной выборке мы получили:
1. Линейная регрессия обучается 19.5 сек. Делает предсказание за 798 мс. RMSE = 2555.
2. CatBoostRegressor обучается за 17.6 сек. Делает предсказание за 449 мс. RMSE = 1580.
3. LGBMRegressor обучается за 19.3 сек. Делает предсказание за 6.62 сек. RMSE = 1630.

Исходя из этого мы вибираем модель CatBoostRegressor - как лидера по времени обучения, предсказания и показателю rmse.

In [72]:
%%time
pr = model_cb.predict(features_test)
print('Лучшее значение rmse и параметров CatBoostRegressor:', rmse(pr, target_test))

Лучшее значение rmse и параметров CatBoostRegressor: 1617.0145409608979
CPU times: user 133 ms, sys: 0 ns, total: 133 ms
Wall time: 131 ms


Показатель и скорость предсказания  на тесте у лучшей модели Catboost - время 131 мс, rmse = 1617. 

In [35]:
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(features_train, target_train)

pr_d =  dummy_regr.predict(features_test)


print('Значение rmse для DummyRegressor:', rmse(pr_d, target_test))

Значение rmse для DummyRegressor: 4541.915865311533
